In [1]:
import os
import re

#!pip install chardet
import chardet
import requests

# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/Colab Notebooks/gothic')
print(os.getcwd())

# Define authors and their corresponding Gutenberg URLs
authors_and_books = {
    "Alexandre Dumas and Auguste Maquet": [
        (
            "The Count of Monte Cristo",
            "https://www.gutenberg.org/ebooks/1184.txt.utf-8",
            24,
            61331,
        )
    ],
    "Algernon Blackwood": [
        (
            "The Empty House and Other Ghost Stories",
            "https://www.gutenberg.org/ebooks/14471.txt.utf-8",
            36,
            6983,
        ),
        (
            "The Wendigo",
            "https://www.gutenberg.org/ebooks/10897.txt.utf-8",
            35,
            1964,
        ),
        (
            "The Willows",
            "https://www.gutenberg.org/ebooks/11438.txt.utf-8",
            25,
            2115,
        ),
        (
            "John Silence, Physician Extraordinary",
            "https://www.gutenberg.org/ebooks/49222.txt.utf-8",
            42,
            11520,
        ),
        (
            "Four Weird Tales",
            "https://www.gutenberg.org/ebooks/16726.txt.utf-8",
            38,
            5847,
        ),
        (
            "Day and Night Stories",
            "https://www.gutenberg.org/ebooks/45964.txt.utf-8",
            42,
            7673,
        ),
        (
            "The Garden of Survival",
            "https://www.gutenberg.org/ebooks/4046.txt.utf-8",
            38,
            2340,
        ),
    ],
    "Ambrose Bierce": [
        (
            "The Damned Thing",
            "https://www.gutenberg.org/ebooks/23172.txt.utf-8",
            33,
            416,
        )
    ],
    "Ann Radcliffe": [
        (
            "The Mysteries of Udolpho",
            "https://www.gutenberg.org/ebooks/3268.txt.utf-8",
            31,
            29925,
        ),
        (
            "The Italian",
            "https://dkan.sfb1288.uni-bielefeld.de/sites/default/files/Ann_Radcliffe-1797-The_Italian-spacy.txt",
            None,
            None,
        ),
        (
            "The Romance of the Forest, interspersed with "
            "some pieces of poetry",
            "https://www.gutenberg.org/ebooks/64701.txt.utf-8",
            23,
            13840,
        ),
        (
            "A Sicilian Romance",
            "https://www.gutenberg.org/ebooks/7371.txt.utf-8",
            37,
            6759,
        ),
    ],
    "Arnold Bennett": [
        (
            "The Ghost: A Modern Fantasy",
            "https://www.gutenberg.org/ebooks/17176.txt.utf-8",
            40,
            7385,
        )
    ],
    "Arthur Conan Doyle": [
        (
            "The Hound of the Baskervilles",
            "https://www.gutenberg.org/ebooks/2852.txt.utf-8",
            28,
            7375,
        ),
        (
            "Tales of Terror and Mystery",
            "https://www.gutenberg.org/ebooks/537.txt.utf-8",
            37,
            7769,
        ),
        (
            "Round the Fire Stories",
            "https://www.gutenberg.org/ebooks/54109.txt.utf-8",
            63,
            10897,
        ),
    ],
    "Arthur Gray": [
        (
            "Tedious brief tales of Granta",
            "https://www.gutenberg.org/ebooks/69438.txt.utf-8",
            44,
            3177,
        )
    ],
    "Arthur Machen": [
        (
            "The Great God Pan",
            "https://www.gutenberg.org/ebooks/389.txt.utf-8",
            27,
            2272,
        ),
        (
            "The House of Souls",
            "https://www.gutenberg.org/ebooks/25016.txt.utf-8",
            39,
            8155,
        ),
        (
            "The Three Impostors; or, The Transmutations",
            "https://www.gutenberg.org/ebooks/35517.txt.utf-8",
            24,
            5588,
        ),
    ],
    "Bram Stoker": [
        (
            "Dracula",
            "https://www.gutenberg.org/ebooks/345.txt.utf-8",
            27,
            15431,
        ),
        (
            "The Lair of the White Worm",
            "https://www.gutenberg.org/ebooks/1188.txt.utf-8",
            35,
            5793,
        ),
        (
            "Dracula's Guest",
            "https://www.gutenberg.org/ebooks/10150.txt.utf-8",
            36,
            5618,
        ),
        (
            "The Man",
            "https://www.gutenberg.org/ebooks/2520.txt.utf-8",
            33,
            10860,
        ),
    ],
    "Catherine Crowe": [
        (
            "The Night-Side of Nature; Or, Ghosts and " "Ghost-Seers",
            "https://www.gutenberg.org/ebooks/54532.txt.utf-8",
            30,
            15809,
        )
    ],
    "Charles Brockden Brown": [
        (
            "Arthur Mervyn; Or, Memoirs of the Year " "1793",
            "https://www.gutenberg.org/ebooks/18508.txt.utf-8",
            35,
            15693,
        ),
        (
            "Ormond; Or, The Secret Witness. Volume " "1 (of 3)",
            "https://www.gutenberg.org/ebooks/36289.txt.utf-8",
            44,
            3238,
        ),
        (
            "Ormond; Or, The Secret Witness. Volume " "2 (of 3)",
            "https://www.gutenberg.org/ebooks/36290.txt.utf-8",
            44,
            3337,
        ),
        (
            "Ormond; Or, The Secret Witness. Volume " "3 (of 3)",
            "https://www.gutenberg.org/ebooks/36291.txt.utf-8",
            44,
            3476,
        ),
    ],
    "Charles Dickens": [
        (
            "A Christmas Carol",
            "https://www.gutenberg.org/ebooks/24022.txt.utf-8",
            44,
            3607,
        ),
        (
            "Three Ghost Stories",
            "https://www.gutenberg.org/ebooks/1289.txt.utf-8",
            32,
            2198,
        ),
        (
            "The Haunted Man and the Ghost's Bargain",
            "https://www.gutenberg.org/ebooks/644.txt.utf-8",
            27,
            3902,
        ),
    ],
    "Charlotte Brontë": [
        (
            "Jane Eyre",
            "https://www.gutenberg.org/ebooks/1260.txt.utf-8",
            29,
            21029,
        ),
        (
            "Villette",
            "https://www.gutenberg.org/ebooks/9182.txt.utf-8",
            27,
            21222,
        ),
    ],
    "Charlotte Perkins Gilman": [
        (
            "The Yellow Wallpaper",
            "https://www.gutenberg.org/ebooks/1952.txt.utf-8",
            27,
            867,
        )
    ],
    "Clara Reeve": [
        (
            "The Old English Baron: a Gothic Story",
            "https://www.gutenberg.org/ebooks/5182.txt.utf-8",
            36,
            6294,
        )
    ],
    "Dorothy Scarborough et al.": [
        (
            "Famous Modern Ghost Stories",
            "https://www.gutenberg.org/ebooks/15143.txt.utf-8",
            46,
            10894,
        )
    ],
    "E. F. Benson": [
        (
            "The room in the tower, and other stories",
            "https://www.gutenberg.org/ebooks/72421.txt.utf-8",
            30,
            8751,
        ),
        (
            "Visible and Invisible",
            "https://www.gutenberg.org/ebooks/60339.txt.utf-8",
            45,
            7478,
        ),
    ],
    "E. G. Swain": [
        (
            "The Stoneground Ghost Tales",
            "https://www.gutenberg.org/ebooks/44581.txt.utf-8",
            48,
            4176,
        )
    ],
    "E. Nesbit": [
        (
            "Grim Tales",
            "https://www.gutenberg.org/ebooks/40321.txt.utf-8",
            48,
            2869,
        )
    ],
    "E. T. A. Hoffmann": [
        (
            "The Devil's Elixir, Vol. 1 (of 2)",
            "https://www.gutenberg.org/ebooks/36494.txt.utf-8",
            44,
            7296,
        ),
        (
            "The Devil's Elixir, Vol. 2 (of 2)",
            "https://www.gutenberg.org/ebooks/37005.txt.utf-8",
            45,
            6522,
        ),
    ],
    "Edgar Allan Poe": [
        (
            "The Works of Edgar Allan Poe",
            "https://www.gutenberg.org/files/2147/2147-0.txt",
            6,
            9572,
        ),
        (
            "The Fall of the House of Usher",
            "https://www.gutenberg.org/ebooks/932.txt.utf-8",
            37,
            825,
        ),
        (
            "The Raven",
            "https://www.gutenberg.org/ebooks/1065.txt.utf-8",
            27,
            159,
        ),
        (
            "The Complete Poetical Works",
            "https://www.gutenberg.org/ebooks/10031.txt.utf-8",
            29,
            9962,
        ),
    ],
    "Eleanor Sleath": [
        (
            "The Orphan of the Rhine",
            "https://gutenberg.net.au/ebooks06/0606561.txt",
            33,
            16202,
        )
    ],
    "Eliza Parsons": [
        (
            "The Castle of Wolfenbach",
            "https://gutenberg.net.au/ebooks06/0606211.txt",
            31,
            8316,
        )
    ],
    "Elizabeth Cleghorn Gaskell": [
        (
            "The Poor Clare",
            "https://www.gutenberg.org/ebooks/2548.txt.utf-8",
            27,
            2113,
        ),
        (
            "The Doom of the Griffiths",
            "https://www.gutenberg.org/ebooks/2549.txt.utf-8",
            27,
            1421,
        ),
    ],
    "Elliott O'Donnell": [
        (
            "Scottish Ghost Stories",
            "https://www.gutenberg.org/ebooks/20034.txt.utf-8",
            54,
            4660,
        )
    ],
    "Emily Brontë": [
        (
            "Wuthering Heights",
            "https://www.gutenberg.org/ebooks/768.txt.utf-8",
            27,
            12368,
        )
    ],
    "Ernest Rhys": [
        (
            "The Haunters & The Haunted",
            "https://www.gutenberg.org/ebooks/17953.txt.utf-8",
            40,
            11664,
        )
    ],
    "F. Marion Crawford": [
        (
            "Wandering Ghosts",
            "https://www.gutenberg.org/ebooks/40386.txt.utf-8",
            62,
            6601,
        ),
        (
            "Man Overboard!",
            "https://www.gutenberg.org/ebooks/24584.txt.utf-8",
            44,
            1728,
        ),
    ],
    "Frances Hodgson Burnett": [
        (
            "The White People",
            "https://www.gutenberg.org/ebooks/459.txt.utf-8",
            33,
            2226,
        )
    ],
    "G. P. R. James": [
        (
            "The Castle of Ehrenstein",
            "https://www.gutenberg.org/ebooks/50325.txt.utf-8",
            60,
            15893,
        )
    ],
    "Gaston Leroux": [
        (
            "The Phantom of the Opera",
            "https://www.gutenberg.org/ebooks/175.txt.utf-8",
            46,
            10446,
        )
    ],
    "George Sylvester Viereck": [
        (
            "The House of the Vampire",
            "https://www.gutenberg.org/ebooks/17144.txt.utf-8",
            31,
            3571,
        )
    ],
    "Gertrude Franklin Horn Atherton": [
        (
            "The Bell in the Fog and Other " "Stories",
            "https://www.gutenberg.org/ebooks/14256.txt.utf-8",
            39,
            6411,
        )
    ],
    "H. P. Lovecraft": [
        (
            "The call of Cthulhu",
            "https://www.gutenberg.org/ebooks/68283.txt.utf-8",
            29,
            1248,
        ),
        (
            "At the mountains of madness",
            "https://www.gutenberg.org/ebooks/70652.txt.utf-8",
            31,
            4648,
        ),
        (
            "The Dunwich Horror",
            "https://www.gutenberg.org/ebooks/50133.txt.utf-8",
            38,
            1750,
        ),
        (
            "The colour out of space",
            "https://www.gutenberg.org/ebooks/68236.txt.utf-8",
            31,
            1163,
        ),
        (
            "The shadow over Innsmouth",
            "https://www.gutenberg.org/ebooks/73181.txt.utf-8",
            31,
            1749,
        ),
        (
            "The case of Charles Dexter Ward",
            "https://www.gutenberg.org/ebooks/73547.txt.utf-8",
            31,
            4465,
        ),
    ],
    "Hawthorne": [
        (
            "The House of the Seven Gables",
            "https://www.gutenberg.org/files/77/77-0.txt",
            28,
            10431,
        )
    ],
    "Heinrich Zschokke": [
        (
            "The Bravo of Venice: A Romance",
            "https://www.gutenberg.org/ebooks/2706.txt.utf-8",
            42,
            4269,
        )
    ],
    "Henry James": [
        (
            "The Turn of the Screw",
            "https://www.gutenberg.org/ebooks/209.txt.utf-8",
            27,
            4572,
        ),
        (
            "The Jolly Corner",
            "https://www.gutenberg.org/ebooks/1190.txt.utf-8",
            36,
            1352,
        ),
    ],
    "Henry Mackenzie": [
        (
            "The Man of Feeling",
            "https://www.gutenberg.org/ebooks/5083.txt.utf-8",
            38,
            3819,
        )
    ],
    "James Hogg": [
        (
            "The Private Memoirs and Confessions of a Justified " "Sinner",
            "https://www.gutenberg.org/ebooks/2276.txt.utf-8",
            39,
            7859,
        )
    ],
    "Jane Austen": [
        (
            "Northanger Abbey",
            "https://www.gutenberg.org/ebooks/121.txt.utf-8",
            27,
            8008,
        )
    ],
    "Jean Toomer": [
        ("Cane", "https://www.gutenberg.org/ebooks/60093.txt.utf-8", 29, 5113)
    ],
    "Jerome K. Jerome": [
        (
            "Told After Supper",
            "https://www.gutenberg.org/ebooks/1993.txt.utf-8",
            35,
            1389,
        )
    ],
    "John Cournos et al.": [
        (
            "The Best British Short Stories of 1922",
            "https://www.gutenberg.org/ebooks/9363.txt.utf-8",
            65,
            12966,
        )
    ],
    "John William Polidori": [
        (
            "The Vampyre",
            "https://www.gutenberg.org/ebooks/6087.txt.utf-8",
            37,
            1272,
        )
    ],
    "Joseph Lewis French": [
        (
            "Great Ghost Stories",
            "https://www.gutenberg.org/ebooks/41170.txt.utf-8",
            42,
            9969,
        ),
        (
            "The Best Psychic Stories",
            "https://www.gutenberg.org/ebooks/36712.txt.utf-8",
            45,
            8845,
        ),
        (
            "Masterpieces of Mystery in Four Volumes: " "Ghost Stories",
            "https://www.gutenberg.org/ebooks/27722.txt.utf-8",
            42,
            6552,
        ),
    ],
    "Joseph Lewis French and Arthur B. Reeve": [
        (
            "The Best Ghost Stories",
            "https://www.gutenberg.org/ebooks/17893.txt.utf-8",
            47,
            8495,
        )
    ],
    "Joseph Sheridan Le Fanu": [
        (
            "The House by the Church-Yard",
            "https://www.gutenberg.org/ebooks/17769.txt.utf-8",
            29,
            23739,
        ),
        (
            "J. S. Le Fanu's Ghostly Tales, Volume " "1",
            "https://www.gutenberg.org/ebooks/11699.txt.utf-8",
            30,
            1683,
        ),
        (
            "Uncle Silas",
            "https://www.gutenberg.org/files/14851/14851-0.txt",
            8,
            19239,
        ),
        (
            "Carmilla",
            "https://www.gutenberg.org/files/10007/10007-0.txt",
            28,
            3340,
        ),
        (
            "In a Glass Darkly, v. 1/3",
            "https://www.gutenberg.org/ebooks/37172.txt.utf-8",
            27,
            4662,
        ),
        (
            "In a Glass Darkly, v. 2/3",
            "https://www.gutenberg.org/ebooks/37173.txt.utf-8",
            24,
            4995,
        ),
        (
            "In a Glass Darkly, v. 3/3",
            "https://www.gutenberg.org/ebooks/37174.txt.utf-8",
            24,
            3975,
        ),
        (
            "Checkmate",
            "https://www.gutenberg.org/ebooks/38460.txt.utf-8",
            53,
            18673,
        ),
        (
            "The Wyvern mystery",
            "https://www.gutenberg.org/ebooks/68569.txt.utf-8",
            37,
            15799,
        ),
        (
            "The Evil Guest",
            "https://www.gutenberg.org/ebooks/10377.txt.utf-8",
            35,
            5034,
        ),
        (
            "Green Tea; Mr. Justice Harbottle",
            "https://www.gutenberg.org/ebooks/11635.txt.utf-8",
            39,
            2978,
        ),
        (
            "Madam Crowl's Ghost and the Dead " "Sexton",
            "https://www.gutenberg.org/ebooks/11610.txt.utf-8",
            36,
            1578,
        ),
        (
            "The Watcher, and other weird stories",
            "https://www.gutenberg.org/ebooks/40510.txt.utf-8",
            51,
            6468,
        ),
    ],
    "Lady Caroline Lamb": [
        (
            "Glenarvon, Volume 1 (of 3)",
            "https://www.gutenberg.org/ebooks/68754.txt.utf-8",
            53,
            4218,
        ),
        (
            "Glenarvon, Volume 2 (of 3)",
            "https://www.gutenberg.org/ebooks/68773.txt.utf-8",
            64,
            5631,
        ),
        (
            "Glenarvon, Volume 3 (of 3)",
            "https://www.gutenberg.org/ebooks/68776.txt.utf-8",
            48,
            4692,
        ),
    ],
    "M. G. Lewis": [
        (
            "The Monk",
            "https://www.gutenberg.org/ebooks/601.txt.utf-8",
            27,
            14500,
        )
    ],
    "M. R. James": [
        (
            "A Warning to the Curious, and Other Ghost Stories",
            "https://www.gutenberg.org/ebooks/66881.txt.utf-8",
            26,
            3490,
        ),
        (
            "Ghost Stories of an Antiquary",
            "https://www.gutenberg.org/ebooks/8486.txt.utf-8",
            29,
            4645,
        ),
        (
            "A Thin Ghost and Others",
            "https://www.gutenberg.org/ebooks/20387.txt.utf-8",
            27,
            2874,
        ),
    ],
    "Marie Belloc Lowndes": [
        (
            "Studies in love and in terror",
            "https://www.gutenberg.org/ebooks/26702.txt.utf-8",
            39,
            6680,
        )
    ],
    "Mary Wollstonecraft Shelley": [
        (
            "Frankenstein",
            "https://www.gutenberg.org/ebooks/84.txt.utf-8",
            28,
            7384,
        ),
        (
            "The Mortal Immortal",
            "https://gutenberg.net.au/ebooks06/0602391.txt",
            33,
            572,
        ),
    ],
    "Maturin": [
        (
            "The Piazza Tales",
            "https://www.gutenberg.org/ebooks/15859.txt.utf-8",
            27,
            8617,
        ),
        (
            "Melmoth the Wanderer, Vol. 1",
            "https://www.gutenberg.org/ebooks/53685.txt.utf-8",
            51,
            5066,
        ),
        (
            "Melmoth the Wanderer, Vol. 2",
            "https://www.gutenberg.org/ebooks/53686.txt.utf-8",
            54,
            4620,
        ),
        (
            "Melmoth the Wanderer, Vol. 3",
            "https://www.gutenberg.org/ebooks/53687.txt.utf-8",
            51,
            5510,
        ),
        (
            "Melmoth the Wanderer, Vol. 4",
            "https://www.gutenberg.org/ebooks/53688.txt.utf-8",
            51,
            6892,
        ),
    ],
    "May Sinclair": [
        (
            "Uncanny Stories",
            "https://www.gutenberg.org/ebooks/59165.txt.utf-8",
            94,
            7857,
        )
    ],
    "Mayne Reid": [
        (
            "The Headless Horseman: A Strange Tale of Texas",
            "https://www.gutenberg.org/ebooks/35587.txt.utf-8",
            42,
            26657,
        )
    ],
    "Miriam Coles Harris": [
        (
            "Rutledge",
            "https://www.gutenberg.org/ebooks/40385.txt.utf-8",
            24,
            17113,
        )
    ],
    "Mrs. Alex. McVeigh Miller": [
        (
            "Flower and Jewel; or, Daisy " "Forrest's Daughter",
            "https://www.gutenberg.org/ebooks/59223.txt.utf-8",
            42,
            7238,
        )
    ],
    "Mrs. Henry Wood": [
        (
            "St. Martin's Eve: A Novel",
            "https://www.gutenberg.org/ebooks/58582.txt.utf-8",
            46,
            21547,
        )
    ],
    "Mrs. Molesworth": [
        (
            "Four Ghost Stories",
            "https://www.gutenberg.org/ebooks/31019.txt.utf-8",
            29,
            3639,
        )
    ],
    "Nathaniel Hawthorne": [
        (
            "The House of the Seven Gables",
            "https://www.gutenberg.org/ebooks/77.txt.utf-8",
            27,
            10430,
        ),
        (
            "The Marble Faun; Or, The Romance of Monte " "Beni - Volume 1",
            "https://www.gutenberg.org/ebooks/2181.txt.utf-8",
            32,
            6616,
        ),
        (
            "The Marble Faun; Or, The Romance of Monte " "Beni - Volume 2",
            "https://www.gutenberg.org/ebooks/2182.txt.utf-8",
            32,
            8114,
        ),
    ],
    "Oliver Onions": [
        (
            "Widdershins ",
            "https://www.gutenberg.org/ebooks/14168.txt.utf-8",
            30,
            8993,
        )
    ],
    "Oscar Wilde": [
        (
            "The Picture of Dorian Gray",
            "https://www.gutenberg.org/ebooks/4078.txt.utf-8",
            38,
            5701,
        ),
        (
            "The Narrative of Arthur Gordon Pym of Nantucket",
            "https://www.gutenberg.org/ebooks/51060.txt.utf-8",
            32,
            6537,
        ),
        (
            "The Happy Prince",
            "https://www.gutenberg.org/ebooks/902.txt.utf-8",
            32,
            1915,
        ),
    ],
    "Prest and Rymer": [
        (
            "Varney the Vampire Or the Feast of Blood",
            "https://www.gutenberg.org/ebooks/14833.txt.utf-8",
            49,
            43310,
        )
    ],
    "Regina Maria Roche": [
        (
            "The Children of the Abbey: A Tale",
            "https://www.gutenberg.org/ebooks/60174.txt.utf-8",
            49,
            24859,
        )
    ],
    "Richard Marsh": [
        (
            "The Beetle: A Mystery",
            "https://www.gutenberg.org/ebooks/5164.txt.utf-8",
            27,
            13768,
        )
    ],
    "Richard Middleton": [
        (
            "The Ghost Ship",
            "https://www.gutenberg.org/ebooks/11045.txt.utf-8",
            34,
            5536,
        )
    ],
    "Robert Louis Stevenson": [
        (
            "The Strange Case of Dr. Jekyll and Mr. " "Hyde",
            "https://www.gutenberg.org/ebooks/42.txt.utf-8",
            28,
            2602,
        )
    ],
    "Robert W. Chambers": [
        (
            "The King in Yellow",
            "https://www.gutenberg.org/ebooks/8492.txt.utf-8",
            42,
            8672,
        )
    ],
    "Rudyard Kipling": [
        (
            "The Phantom Rickshaw, and Other Ghost Stories",
            "https://www.gutenberg.org/ebooks/2806.txt.utf-8",
            33,
            4735,
        )
    ],
    "S. Baring-Gould": [
        (
            "The Best Ghost Stories",
            "https://www.gutenberg.org/ebooks/36638.txt.utf-8",
            29,
            14356,
        )
    ],
    "T. Smollett": [
        (
            "The Adventures of Ferdinand Count Fathom — " "Complete",
            "https://www.gutenberg.org/ebooks/6761.txt.utf-8",
            27,
            15921,
        )
    ],
    "Thomas Holcroft": [
        (
            "Anna St. Ives",
            "https://www.gutenberg.org/ebooks/9468.txt.utf-8",
            39,
            20612,
        )
    ],
    "VERNON LEE": [
        (
            "Penelope Brandling: A Tale of the Welsh coast in "
            "the Eighteenth Century",
            "https://www.gutenberg.org/ebooks/37180.txt.utf-8",
            34,
            2193,
        )
    ],
    "Victor Hugo": [
        (
            "Notre-Dame de Paris",
            "https://www.gutenberg.org/ebooks/2610.txt.utf-8",
            34,
            22834,
        )
    ],
    "W. Bob Holland": [
        (
            "Twenty-Five Ghost Stories",
            "https://www.gutenberg.org/ebooks/53419.txt.utf-8",
            43,
            5970,
        )
    ],
    "W. T. Stead": [
        (
            "Real Ghost Stories",
            "https://www.gutenberg.org/ebooks/20420.txt.utf-8",
            37,
            6493,
        )
    ],
    "W. W. Jacobs": [
        (
            "The Well",
            "https://www.gutenberg.org/ebooks/12124.txt.utf-8",
            32,
            621,
        )
    ],
    "Walpole": [
        (
            "The Castle of Otranto",
            "https://www.gutenberg.org/ebooks/696.txt.utf-8",
            38,
            4149,
        )
    ],
    "Walter Scott": [
        (
            "The Tapestried Chamber, and Death of the Laird's " "Jock",
            "https://www.gutenberg.org/ebooks/1668.txt.utf-8",
            33,
            834,
        )
    ],
    "Washington Irving": [
        (
            "The Legend of Sleepy Hollow",
            "https://www.gutenberg.org/ebooks/41.txt.utf-8",
            27,
            1152,
        )
    ],
    "Wilkie Collins": [
        (
            "The Woman in White",
            "https://www.gutenberg.org/ebooks/583.txt.utf-8",
            26,
            25911,
        ),
        (
            "The Dead Secret: A Novel",
            "https://www.gutenberg.org/ebooks/43092.txt.utf-8",
            47,
            14702,
        ),
    ],
    "William Beckford": [
        (
            "Vathek; An Arabian Tale",
            "https://www.gutenberg.org/ebooks/42401.txt.utf-8",
            33,
            4004,
        )
    ],
    "William Godwin": [
        (
            "St. Leon: A Tale of the Sixteenth Century",
            "https://www.gutenberg.org/ebooks/53707.txt.utf-8",
            49,
            17325,
        )
    ],
    "William Hope Hodgson": [
        (
            "Carnacki, the Ghost Finder",
            "https://www.gutenberg.org/ebooks/10832.txt.utf-8",
            35,
            5176,
        )
    ],
}

base_path = "./gothic_novels"


def make_safe_filename(s, max_length=255):
    s = re.sub(r'[\/:*?"<>|;,.]', "", s)
    s = s.strip()
    if len(s) > max_length:
        s = s[:max_length]
    s = s.replace(" ", "_")
    s = s.replace("'", "_")
    return s


# Create directories for each author
for author in authors_and_books.keys():
    author = make_safe_filename(author)
    author_path = os.path.join(base_path, author)
    os.makedirs(author_path, exist_ok=True)

# Download text files using requests
for author, books in authors_and_books.items():
    author = make_safe_filename(author)
    for book_title, book_url, start, stop in books:
        book_title = make_safe_filename(book_title)
        book_path = os.path.join(base_path, author, f"{book_title}.txt")
        if not os.path.exists(book_path):
            response = requests.get(book_url)
            if response.status_code == 200:
                # Detect encoding
                detected_encoding = chardet.detect(response.content)["encoding"]
                print(
                    f"Detected encoding for {book_title}: {detected_encoding}"
                )

                if detected_encoding is not None:
                    # Use the detected encoding
                    content = response.content.decode(detected_encoding)
                else:
                    # Fallback if encoding detection fails
                    content = response.text

                # Write the content to file using UTF-8 encoding
                with open(
                    book_path, "w", encoding="utf-8", errors="ignore"
                ) as f:
                    f.write(content)

                print(f"Downloaded {book_title} for author {author}")
            else:
                print(f"Failed to download {book_title} for author {author}")
        else:
            print(f"{book_title} already downloaded for author {author}")
print("Download complete!")

# Verify that directories and files exist
for author, books in authors_and_books.items():
    author = make_safe_filename(author)
    author_path = os.path.join(base_path, author)
    if not os.path.exists(author_path):
        print(f"Directory does not exist: {author_path}")
    else:
        files = os.listdir(author_path)
        if not files:
            print(f"No files found in directory: {author_path}")
        else:
            print(f"Found {len(files)} files in directory: {author_path}")


import unicodedata


def normalize_text(text):
    # Replace curly quotes with straight quotes
    text = re.sub(
        r"[‘’]", "'", text
    )  # Replace left and right single quotes with '
    text = re.sub(
        r"[“”]", '"', text
    )  # Replace left and right double quotes with "

    # Replace en dash and em dash with a hyphen or double hyphen
    text = re.sub(r"–", "-", text)  # Replace en dash with hyphen
    text = re.sub(r"—", "--", text)  # Replace em dash with double hyphen

    # Replace ellipsis with three dots
    text = re.sub(r"…", "...", text)

    # Replace other non-ASCII punctuation with ASCII equivalents
    text = re.sub(r"«", "<<", text)  # Replace left double angle quote with <<
    text = re.sub(r"»", ">>", text)  # Replace right double angle quote with >>
    text = re.sub(r"§", "SS", text)  # Replace section sign with SS
    text = re.sub(r"•", "*", text)  # Replace bullet with asterisk
    text = re.sub(r"‽", "?!", text)  # Replace interrobang with ?!
    text = re.sub(r"′", "'", text)  # Replace prime with single quote
    text = re.sub(r"″", '"', text)  # Replace double prime with double quote
    return (
        unicodedata.normalize("NFKD", text)
        .encode("ascii", "ignore")
        .decode("ascii")
    )


for author, books in authors_and_books.items():
    author = make_safe_filename(author)
    for book_title, book_url, start, stop in books:
        book_title = make_safe_filename(book_title)
        book_path = os.path.join(base_path, author, f"{book_title}.txt")
        if not os.path.exists(book_path):
            print(f"Could not find {book_title} for author {author}")
        else:
            print(f"Found {book_title} for author {author}")
            trimmed_book_path = os.path.join(
                base_path, author, f"{book_title}_trimmed.txt"
            )
            if os.path.exists(trimmed_book_path):
                os.remove(trimmed_book_path)
            with open(book_path, "r") as book, open(
                trimmed_book_path, "w"
            ) as trimmed_book:
                print(f"Saving {book_title}_trimmed.txt")
                for i, line in enumerate(book):
                    if start != None and stop != None:
                        if i < start:
                            continue
                        if i > stop:
                            break
                    # Write the line to the new file
                    trimmed_book.write(normalize_text(line))


# Aggregate texts
all_texts = []
for author in authors_and_books.keys():
    author = make_safe_filename(author)
    author_path = os.path.join(base_path, author)
    for filename in os.listdir(author_path):
        if filename.endswith("_trimmed.txt"):
            with open(
                os.path.join(author_path, filename), "r", encoding="utf-8"
            ) as file:
                text = file.read()
                all_texts.append("<|startoftext|>" + text  + "<|endoftext|>")

# Combine texts into a single file
combined_texts = "".join(all_texts)
with open("gothic_novels_combined.txt", "w", encoding="utf-8") as outfile:
    outfile.write(normalize_text(combined_texts))

print("Gothic novels dataset created successfully!")

/content
Detected encoding for The_Count_of_Monte_Cristo: UTF-8-SIG
Downloaded The_Count_of_Monte_Cristo for author Alexandre_Dumas_and_Auguste_Maquet
Detected encoding for The_Empty_House_and_Other_Ghost_Stories: UTF-8-SIG
Downloaded The_Empty_House_and_Other_Ghost_Stories for author Algernon_Blackwood
Detected encoding for The_Wendigo: UTF-8-SIG
Downloaded The_Wendigo for author Algernon_Blackwood
Detected encoding for The_Willows: UTF-8-SIG
Downloaded The_Willows for author Algernon_Blackwood
Detected encoding for John_Silence_Physician_Extraordinary: UTF-8-SIG
Downloaded John_Silence_Physician_Extraordinary for author Algernon_Blackwood
Detected encoding for Four_Weird_Tales: UTF-8-SIG
Downloaded Four_Weird_Tales for author Algernon_Blackwood
Detected encoding for Day_and_Night_Stories: UTF-8-SIG
Downloaded Day_and_Night_Stories for author Algernon_Blackwood
Detected encoding for The_Garden_of_Survival: UTF-8-SIG
Downloaded The_Garden_of_Survival for author Algernon_Blackwood
Detec

In [ ]:
def print_sorted_dict(d):
    # Sort the dictionary by keys
    sorted_keys = sorted(d.keys())

    # Print the dictionary literal
    print("{")
    for key in sorted_keys:
        print(f"\"{key}\":\n   {repr(d[key])},\n")
    print("}")

# Example usage:
authors_and_books = {
    "Walpole": [
        ("The Castle of Otranto", "https://www.gutenberg.org/ebooks/696.txt.utf-8", 38, 4149)
    ],
    "Eleanor Sleath": [
        ("The Orphan of the Rhine", "https://gutenberg.net.au/ebooks06/0606561.txt", 33, 16202)
    ],
    "Ann Radcliffe": [
        ("The Mysteries of Udolpho", "https://www.gutenberg.org/ebooks/3268.txt.utf-8", 31, 29925),
        ("The Italian", "https://dkan.sfb1288.uni-bielefeld.de/sites/default/files/Ann_Radcliffe-1797-The_Italian-spacy.txt", None,  None),
        ("The Romance of the Forest, interspersed with some pieces of poetry", "https://www.gutenberg.org/ebooks/64701.txt.utf-8", 23,  13840),
        ("A Sicilian Romance", "https://www.gutenberg.org/ebooks/7371.txt.utf-8", 37,  6759)
    ],
    "M. G. Lewis": [
        ("The Monk", "https://www.gutenberg.org/ebooks/601.txt.utf-8", 27, 14500)
    ],
    "Mary Wollstonecraft Shelley": [
        ("Frankenstein", "https://www.gutenberg.org/ebooks/84.txt.utf-8", 28, 7384),
        ("The Mortal Immortal", "https://gutenberg.net.au/ebooks06/0602391.txt", 33, 572)
    ],
    "Bram Stoker": [
        ("Dracula", "https://www.gutenberg.org/ebooks/345.txt.utf-8", 27, 15431),
        ("The Lair of the White Worm", "https://www.gutenberg.org/ebooks/1188.txt.utf-8",  35, 5793),
        ("Dracula's Guest", "https://www.gutenberg.org/ebooks/10150.txt.utf-8",  36, 5618),
        ("The Man", "https://www.gutenberg.org/ebooks/2520.txt.utf-8",  33, 10860)
    ],
    "Emily Brontë": [
        ("Wuthering Heights", "https://www.gutenberg.org/ebooks/768.txt.utf-8", 27, 12368)
    ],
    "Charlotte Brontë": [
        ("Jane Eyre", "https://www.gutenberg.org/ebooks/1260.txt.utf-8", 29, 21029),
        ("Villette", "https://www.gutenberg.org/ebooks/9182.txt.utf-8", 27, 21222)
    ],
    "Maturin": [
        ("The Piazza Tales", "https://www.gutenberg.org/ebooks/15859.txt.utf-8", 27, 8617),
        ("Melmoth the Wanderer, Vol. 1", "https://www.gutenberg.org/ebooks/53685.txt.utf-8", 51, 5066),
        ("Melmoth the Wanderer, Vol. 2", "https://www.gutenberg.org/ebooks/53686.txt.utf-8", 54, 4620),
        ("Melmoth the Wanderer, Vol. 3", "https://www.gutenberg.org/ebooks/53687.txt.utf-8", 51, 5510),
        ("Melmoth the Wanderer, Vol. 4", "https://www.gutenberg.org/ebooks/53688.txt.utf-8", 51, 6892),
    ],
    "Hawthorne": [
        ("The House of the Seven Gables", "https://www.gutenberg.org/files/77/77-0.txt", 28,  10431)
    ],
    "Robert Louis Stevenson": [
        ("The Strange Case of Dr. Jekyll and Mr. Hyde", "https://www.gutenberg.org/ebooks/42.txt.utf-8", 28,  2602)
    ],
    "Clara Reeve": [
        ("The Old English Baron: a Gothic Story", "https://www.gutenberg.org/ebooks/5182.txt.utf-8", 36,  6294)
    ],
    "Victor Hugo": [
        ("Notre-Dame de Paris", "https://www.gutenberg.org/ebooks/2610.txt.utf-8", 34,  22834)
    ],
    "Edgar Allan Poe": [
        ("The Works of Edgar Allan Poe", "https://www.gutenberg.org/files/2147/2147-0.txt", 6, 9572),
        ("The Fall of the House of Usher", "https://www.gutenberg.org/ebooks/932.txt.utf-8", 37,  825),
        ("The Raven", "https://www.gutenberg.org/ebooks/1065.txt.utf-8", 27,  159),
        ("The Complete Poetical Works", "https://www.gutenberg.org/ebooks/10031.txt.utf-8", 29,  9962)
    ],
    "Prest and Rymer": [
        ("Varney the Vampire Or the Feast of Blood", "https://www.gutenberg.org/ebooks/14833.txt.utf-8", 49,  43310)
    ],
    "Joseph Sheridan Le Fanu": [
        ("The House by the Church-Yard", "https://www.gutenberg.org/ebooks/17769.txt.utf-8", 29,  23739),
        ("J. S. Le Fanu's Ghostly Tales, Volume 1", "https://www.gutenberg.org/ebooks/11699.txt.utf-8", 30,  1683),
        ("Uncle Silas", "https://www.gutenberg.org/files/14851/14851-0.txt", 8, 19239),
        ("Carmilla", "https://www.gutenberg.org/files/10007/10007-0.txt", 28, 3340),
        ("In a Glass Darkly, v. 1/3", "https://www.gutenberg.org/ebooks/37172.txt.utf-8", 27, 4662),
        ("In a Glass Darkly, v. 2/3", "https://www.gutenberg.org/ebooks/37173.txt.utf-8", 24, 4995),
        ("In a Glass Darkly, v. 3/3", "https://www.gutenberg.org/ebooks/37174.txt.utf-8", 24, 3975),
        ("Checkmate", "https://www.gutenberg.org/ebooks/38460.txt.utf-8", 53, 18673),
        ("The Wyvern mystery", "https://www.gutenberg.org/ebooks/68569.txt.utf-8", 37, 15799),
        ("The Evil Guest", "https://www.gutenberg.org/ebooks/10377.txt.utf-8", 35, 5034),
        ("Green Tea; Mr. Justice Harbottle", "https://www.gutenberg.org/ebooks/11635.txt.utf-8", 39, 2978),
        ("Madam Crowl's Ghost and the Dead Sexton", "https://www.gutenberg.org/ebooks/11610.txt.utf-8", 36, 1578),
        ("The Watcher, and other weird stories", "https://www.gutenberg.org/ebooks/40510.txt.utf-8", 51, 6468)
    ],
    "Wilkie Collins": [
        ("The Woman in White", "https://www.gutenberg.org/ebooks/583.txt.utf-8", 26,  25911),
        ("The Dead Secret: A Novel", "https://www.gutenberg.org/ebooks/43092.txt.utf-8", 47,  14702)
    ],
    "Oscar Wilde": [
        ("The Picture of Dorian Gray", "https://www.gutenberg.org/ebooks/4078.txt.utf-8", 38,  5701),
        ("The Narrative of Arthur Gordon Pym of Nantucket", "https://www.gutenberg.org/ebooks/51060.txt.utf-8", 32,  6537),
        ("The Happy Prince", "https://www.gutenberg.org/ebooks/902.txt.utf-8", 32,  1915)
    ],
    "Henry James": [
        ("The Turn of the Screw", "https://www.gutenberg.org/ebooks/209.txt.utf-8", 27,  4572),
        ("The Jolly Corner", "https://www.gutenberg.org/ebooks/1190.txt.utf-8", 36,  1352)
    ],
    "Gaston Leroux": [
        ("The Phantom of the Opera", "https://www.gutenberg.org/ebooks/175.txt.utf-8", 46,  10446)
    ],
    "Arthur Conan Doyle": [
        ("The Hound of the Baskervilles", "https://www.gutenberg.org/ebooks/2852.txt.utf-8", 28,  7375),
        ("Tales of Terror and Mystery", "https://www.gutenberg.org/ebooks/537.txt.utf-8", 37,  7769),
        ("Round the Fire Stories", "https://www.gutenberg.org/ebooks/54109.txt.utf-8", 63,  10897)
    ],
    "Richard Marsh": [
        ("The Beetle: A Mystery", "https://www.gutenberg.org/ebooks/5164.txt.utf-8", 27,  13768)
    ],
    "John William Polidori": [
        ("The Vampyre", "https://www.gutenberg.org/ebooks/6087.txt.utf-8", 37,  1272)
    ],
    "Arthur Machen": [
        ("The Great God Pan", "https://www.gutenberg.org/ebooks/389.txt.utf-8", 27,  2272),
        ("The House of Souls", "https://www.gutenberg.org/ebooks/25016.txt.utf-8", 39,  8155),
        ("The Three Impostors; or, The Transmutations", "https://www.gutenberg.org/ebooks/35517.txt.utf-8", 24,  5588)
    ],
    "Jane Austen": [
        ("Northanger Abbey", "https://www.gutenberg.org/ebooks/121.txt.utf-8", 27,  8008)
    ],
    "James Hogg": [
        ("The Private Memoirs and Confessions of a Justified Sinner", "https://www.gutenberg.org/ebooks/2276.txt.utf-8", 39,  7859)
    ],
    "Charles Dickens": [
        ("A Christmas Carol", "https://www.gutenberg.org/ebooks/24022.txt.utf-8", 44,  3607),
        ("Three Ghost Stories", "https://www.gutenberg.org/ebooks/1289.txt.utf-8", 32,  2198),
        ("The Haunted Man and the Ghost's Bargain", "https://www.gutenberg.org/ebooks/644.txt.utf-8", 27,  3902)

    ],
    "Catherine Crowe": [
        ("The Night-Side of Nature; Or, Ghosts and Ghost-Seers", "https://www.gutenberg.org/ebooks/54532.txt.utf-8", 30,  15809)
    ],
    "Nathaniel Hawthorne": [
        ("The House of the Seven Gables", "https://www.gutenberg.org/ebooks/77.txt.utf-8", 27,  10430),
        ("The Marble Faun; Or, The Romance of Monte Beni - Volume 1", "https://www.gutenberg.org/ebooks/2181.txt.utf-8", 32,  6616),
        ("The Marble Faun; Or, The Romance of Monte Beni - Volume 2", "https://www.gutenberg.org/ebooks/2182.txt.utf-8", 32,  8114),
    ],
    "Charlotte Perkins Gilman": [
        ("The Yellow Wallpaper", "https://www.gutenberg.org/ebooks/1952.txt.utf-8", 27,  867)
    ],
    "M. R. James": [
        ("A Warning to the Curious, and Other Ghost Stories", "https://www.gutenberg.org/ebooks/66881.txt.utf-8", 26,  3490),
        ("Ghost Stories of an Antiquary", "https://www.gutenberg.org/ebooks/8486.txt.utf-8", 29,  4645),
        ("A Thin Ghost and Others", "https://www.gutenberg.org/ebooks/20387.txt.utf-8", 27,  2874)
    ],
    "Charles Brockden Brown": [
        ("Arthur Mervyn; Or, Memoirs of the Year 1793", "https://www.gutenberg.org/ebooks/18508.txt.utf-8", 35,  15693),
        ("Ormond; Or, The Secret Witness. Volume 1 (of 3)", "https://www.gutenberg.org/ebooks/36289.txt.utf-8", 44,  3238),
        ("Ormond; Or, The Secret Witness. Volume 2 (of 3)", "https://www.gutenberg.org/ebooks/36290.txt.utf-8", 44,  3337),
        ("Ormond; Or, The Secret Witness. Volume 3 (of 3)", "https://www.gutenberg.org/ebooks/36291.txt.utf-8", 44,  3476)
    ],
    "Eliza Parsons": [
        ("The Castle of Wolfenbach", "https://gutenberg.net.au/ebooks06/0606211.txt", 31,  8316)
    ],
    "Jean Toomer": [
        ("Cane", "https://www.gutenberg.org/ebooks/60093.txt.utf-8", 29,  5113)
    ],
    "Alexandre Dumas and Auguste Maquet": [
        ("The Count of Monte Cristo", "https://www.gutenberg.org/ebooks/1184.txt.utf-8", 24,  61331)
    ],
    "W. Bob Holland": [
        ("Twenty-Five Ghost Stories", "https://www.gutenberg.org/ebooks/53419.txt.utf-8", 43,  5970)
    ],
    "Rudyard Kipling": [
        ("The Phantom Rickshaw, and Other Ghost Stories", "https://www.gutenberg.org/ebooks/2806.txt.utf-8", 33,  4735)
    ],
    "Algernon Blackwood": [
        ("The Empty House and Other Ghost Stories", "https://www.gutenberg.org/ebooks/14471.txt.utf-8", 36,  6983),
        ("The Wendigo", "https://www.gutenberg.org/ebooks/10897.txt.utf-8", 35,  1964),
        ("The Willows", "https://www.gutenberg.org/ebooks/11438.txt.utf-8", 25,  2115),
        ("John Silence, Physician Extraordinary", "https://www.gutenberg.org/ebooks/49222.txt.utf-8", 42,  11520),
        ("Four Weird Tales", "https://www.gutenberg.org/ebooks/16726.txt.utf-8", 38,  5847),
        ("Day and Night Stories", "https://www.gutenberg.org/ebooks/45964.txt.utf-8", 42,  7673),
        ("The Garden of Survival", "https://www.gutenberg.org/ebooks/4046.txt.utf-8", 38,  2340)
    ],
    "Dorothy Scarborough et al.": [
        ("Famous Modern Ghost Stories", "https://www.gutenberg.org/ebooks/15143.txt.utf-8", 46,  10894)
    ],
    "John Cournos et al.": [
        ("The Best British Short Stories of 1922", "https://www.gutenberg.org/ebooks/9363.txt.utf-8", 65,  12966)
    ],
    "Arnold Bennett": [
        ("The Ghost: A Modern Fantasy", "https://www.gutenberg.org/ebooks/17176.txt.utf-8", 40,  7385)
    ],
    "E. F. Benson": [
        ("The room in the tower, and other stories", "https://www.gutenberg.org/ebooks/72421.txt.utf-8", 30,  8751),
        ("Visible and Invisible", "https://www.gutenberg.org/ebooks/60339.txt.utf-8", 45,  7478)
    ],
    "Elliott O'Donnell": [
        ("Scottish Ghost Stories", "https://www.gutenberg.org/ebooks/20034.txt.utf-8", 54,  4660)
    ],
    "H. P. Lovecraft": [
        ("The call of Cthulhu", "https://www.gutenberg.org/ebooks/68283.txt.utf-8", 29,  1248),
        ("At the mountains of madness", "https://www.gutenberg.org/ebooks/70652.txt.utf-8", 31,  4648),
        ("The Dunwich Horror", "https://www.gutenberg.org/ebooks/50133.txt.utf-8", 38,  1750),
        ("The colour out of space", "https://www.gutenberg.org/ebooks/68236.txt.utf-8", 31,  1163),
        ("The shadow over Innsmouth", "https://www.gutenberg.org/ebooks/73181.txt.utf-8", 31,  1749),
        ("The case of Charles Dexter Ward", "https://www.gutenberg.org/ebooks/73547.txt.utf-8", 31,  4465)
    ],
    "George Sylvester Viereck": [
        ("The House of the Vampire", "https://www.gutenberg.org/ebooks/17144.txt.utf-8", 31,  3571)
    ],
    "Ambrose Bierce": [
        ("The Damned Thing", "https://www.gutenberg.org/ebooks/23172.txt.utf-8", 33,  416)
    ],
    "Robert W. Chambers": [
        ("The King in Yellow", "https://www.gutenberg.org/ebooks/8492.txt.utf-8", 42,  8672)
    ],
    "Joseph Lewis French and Arthur B. Reeve": [
        ("The Best Ghost Stories", "https://www.gutenberg.org/ebooks/17893.txt.utf-8", 47,  8495)
    ],
    "S. Baring-Gould": [
        ("The Best Ghost Stories", "https://www.gutenberg.org/ebooks/36638.txt.utf-8", 29,  14356)
    ],
    "Richard Middleton": [
        ("The Ghost Ship", "https://www.gutenberg.org/ebooks/11045.txt.utf-8", 34,  5536)
    ],
    "Joseph Lewis French": [
        ("Great Ghost Stories", "https://www.gutenberg.org/ebooks/41170.txt.utf-8", 42,  9969),
        ("The Best Psychic Stories", "https://www.gutenberg.org/ebooks/36712.txt.utf-8", 45,  8845),
        ("Masterpieces of Mystery in Four Volumes: Ghost Stories", "https://www.gutenberg.org/ebooks/27722.txt.utf-8", 42,  6552)
    ],
    "Washington Irving": [
        ("The Legend of Sleepy Hollow", "https://www.gutenberg.org/ebooks/41.txt.utf-8", 27,  1152)
    ],
    "William Beckford": [
        ("Vathek; An Arabian Tale", "https://www.gutenberg.org/ebooks/42401.txt.utf-8", 33,  4004)
    ],
    "T. Smollett": [
        ("The Adventures of Ferdinand Count Fathom — Complete", "https://www.gutenberg.org/ebooks/6761.txt.utf-8", 27,  15921)
    ],
    "Henry Mackenzie": [
        ("The Man of Feeling", "https://www.gutenberg.org/ebooks/5083.txt.utf-8", 38,  3819)
    ],
    "VERNON LEE": [
        ("Penelope Brandling: A Tale of the Welsh coast in the Eighteenth Century", "https://www.gutenberg.org/ebooks/37180.txt.utf-8", 34,  2193)
    ],
    "William Godwin": [
        ("St. Leon: A Tale of the Sixteenth Century", "https://www.gutenberg.org/ebooks/53707.txt.utf-8", 49,  17325)
    ],
    "Gertrude Franklin Horn Atherton": [
        ("The Bell in the Fog and Other Stories", "https://www.gutenberg.org/ebooks/14256.txt.utf-8", 39,  6411)
    ],
    "Regina Maria Roche": [
        ("The Children of the Abbey: A Tale", "https://www.gutenberg.org/ebooks/60174.txt.utf-8", 49,  24859)
    ],
    "Heinrich Zschokke": [
        ("The Bravo of Venice: A Romance", "https://www.gutenberg.org/ebooks/2706.txt.utf-8", 42,  4269)
    ],
    "E. T. A. Hoffmann": [
        ("The Devil's Elixir, Vol. 1 (of 2)", "https://www.gutenberg.org/ebooks/36494.txt.utf-8", 44,  7296),
        ("The Devil's Elixir, Vol. 2 (of 2)", "https://www.gutenberg.org/ebooks/37005.txt.utf-8", 45,  6522)
    ],
    "Mrs. Alex. McVeigh Miller": [
        ("Flower and Jewel; or, Daisy Forrest's Daughter", "https://www.gutenberg.org/ebooks/59223.txt.utf-8", 42,  7238)
    ],
    "G. P. R. James": [
        ("The Castle of Ehrenstein", "https://www.gutenberg.org/ebooks/50325.txt.utf-8", 60,  15893)
    ],
    "Lady Caroline Lamb": [
        ("Glenarvon, Volume 1 (of 3)", "https://www.gutenberg.org/ebooks/68754.txt.utf-8", 53,  4218),
        ("Glenarvon, Volume 2 (of 3)", "https://www.gutenberg.org/ebooks/68773.txt.utf-8", 64,  5631),
        ("Glenarvon, Volume 3 (of 3)", "https://www.gutenberg.org/ebooks/68776.txt.utf-8", 48,  4692)
    ],
    "Mrs. Henry Wood": [
        ("St. Martin's Eve: A Novel", "https://www.gutenberg.org/ebooks/58582.txt.utf-8", 46,  21547)
    ],
    "Thomas Holcroft": [
        ("Anna St. Ives", "https://www.gutenberg.org/ebooks/9468.txt.utf-8", 39,  20612)
    ],
    "Miriam Coles Harris": [
        ("Rutledge", "https://www.gutenberg.org/ebooks/40385.txt.utf-8", 24,  17113)
    ],
    "F. Marion Crawford": [
        ("Wandering Ghosts", "https://www.gutenberg.org/ebooks/40386.txt.utf-8", 62,  6601),
        ("Man Overboard!", "https://www.gutenberg.org/ebooks/24584.txt.utf-8", 44,  1728)
    ],
    "Mayne Reid": [
        ("The Headless Horseman: A Strange Tale of Texas", "https://www.gutenberg.org/ebooks/35587.txt.utf-8", 42,  26657)
    ],
    "Walter Scott": [
        ("The Tapestried Chamber, and Death of the Laird's Jock", "https://www.gutenberg.org/ebooks/1668.txt.utf-8", 33,  834)
    ],
    "Ernest Rhys": [
        ("The Haunters & The Haunted", "https://www.gutenberg.org/ebooks/17953.txt.utf-8", 40,  11664)
    ],
    "W. T. Stead": [
        ("Real Ghost Stories", "https://www.gutenberg.org/ebooks/20420.txt.utf-8", 37,  6493)
    ],
    "Frances Hodgson Burnett": [
        ("The White People", "https://www.gutenberg.org/ebooks/459.txt.utf-8", 33,  2226)
    ],
    "E. Nesbit": [
        ("Grim Tales", "https://www.gutenberg.org/ebooks/40321.txt.utf-8", 48,  2869)
    ],
    "May Sinclair": [
        ("Uncanny Stories", "https://www.gutenberg.org/ebooks/59165.txt.utf-8", 94,  7857)
    ],
    "Oliver Onions": [
        ("Widdershins ", "https://www.gutenberg.org/ebooks/14168.txt.utf-8", 30,  8993)
    ],
    "Jerome K. Jerome": [
        ("Told After Supper", "https://www.gutenberg.org/ebooks/1993.txt.utf-8", 35,  1389)
    ],
    "E. G. Swain": [
        ("The Stoneground Ghost Tales", "https://www.gutenberg.org/ebooks/44581.txt.utf-8", 48,  4176)
    ],
    "Arthur Gray": [
        ("Tedious brief tales of Granta", "https://www.gutenberg.org/ebooks/69438.txt.utf-8", 44,  3177)
    ],
    "Mrs. Molesworth": [
        ("Four Ghost Stories", "https://www.gutenberg.org/ebooks/31019.txt.utf-8", 29,  3639)
    ],
    "William Hope Hodgson": [
        ("Carnacki, the Ghost Finder", "https://www.gutenberg.org/ebooks/10832.txt.utf-8", 35,  5176)
    ],
    "Elizabeth Cleghorn Gaskell": [
        ("The Poor Clare", "https://www.gutenberg.org/ebooks/2548.txt.utf-8", 27,  2113),
        ("The Doom of the Griffiths", "https://www.gutenberg.org/ebooks/2549.txt.utf-8", 27,  1421)
    ],
    "W. W. Jacobs": [
        ("The Well", "https://www.gutenberg.org/ebooks/12124.txt.utf-8", 32,  621)
    ],
    "Marie Belloc Lowndes": [
        ("Studies in love and in terror", "https://www.gutenberg.org/ebooks/26702.txt.utf-8", 39,  6680)
    ]
}

#print_sorted_dict(authors_and_books)

import pprint

def pretty_print_dict(d):
    # Use pprint to pretty-print the dictionary
    pprint.pprint(d, sort_dicts=True, width=80, indent=4)

pretty_print_dict(authors_and_books)

In [ ]:
import pprint

def pretty_print_dict(d):
    # Use pprint to pretty-print the dictionary
    pprint.pprint(d, sort_dicts=True, width=80, indent=4)

# Example usage:
example_dict = {
    "banana": 3,
    "apple": 1,
    "cherry": {
        "red": 2,
        "dark_red": 3
    },
    "date": [4, 5, 6],
    "elderberry": {
        "color": "purple",
        "size": "small"
    }
}

pretty_print_dict(example_dict)


{   'apple': 1,
    'banana': 3,
    'cherry': {'dark_red': 3, 'red': 2},
    'date': [4, 5, 6],
    'elderberry': {'color': 'purple', 'size': 'small'}}


In [ ]:
import unicodedata

def normalize_text(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

print(normalize_text("Marseilles—The Arrival"))
unicodedata.normalize('NFKD', "Marseilles—The Arrival")

MarseillesThe Arrival


'Marseilles—The Arrival'